In [1]:
import sys
import os
# Navigate up one level to the parent directory and append it to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import nfl_data_py as nfl
import pandas as pd
from src import utils

## Questions to answer
* Who was the best picker over the period of time?
* What trends emerged (picking home/away, dogs/favorites)
* Were we homers? (picking our teams, picking the same teams)
* Was there correlation between doing well one week, and doing well the next week?
* Did the different lines matter?

In [2]:
teams = utils.get_nfl_teams()

In [37]:
def parse_picks(season, week):
    df = pd.read_excel(f'../../data/betting/homers-{season}.xlsx', sheet_name=f'Wk {week}', usecols=range(14), header=1)
    df = df.drop(df.index[9:])
    df = df.dropna(axis=1, how='all')
    if 'Notes' in df.columns:
        df = df.drop(columns=['Notes'])
    df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])
    df = utils.clean_df_columns(df)
    df = df.rename(columns={'team': 'final', 'picks': 'pick_type'})
    assert all(col in ['pick_type', 'final', 'ben', 'chuck', 'griffin', 'harry', 'hunter'] for col in df.columns)
    assert len(df) == 9
    df['pick_type'] = df.pick_type.map({i: 'reg' for i in range(1, 6)}).fillna(df['pick_type']).apply(str.lower)
    df['pick_type'] = df.pick_type.map({'underdog': 'ud', 'survivor': 'sd'}).fillna(df['pick_type'])
    df['pick_type'] = df.pick_type.apply(lambda x: 'reg' if x not in ['bb', 'reg', 'sd', 'ud', 'mnf'] else x)

    transformed = pd.melt(df, id_vars=['pick_type'], var_name='picker', value_name='pick').dropna()
    transformed['season'] = season
    transformed['week'] = week
    transformed['pick'] = transformed['pick'].apply(str.upper).apply(str.strip).apply(lambda x: x.split('/')[0] if '/' in x else x)
    transformed = transformed[transformed['pick'] != '']
    # Map bad team names
    transformed['pick'] = transformed['pick'].map({
        'LAR': 'LA', 
        'WSH': 'WAS', 
        'WFT': 'WAS',
        'JAG': 'JAX',
        'JAC': 'JAX',
        'PHL': 'PHI',
    }).fillna(transformed['pick'])

    bad_teams = [team for team in transformed['pick'].unique() if team not in teams]
    assert len(bad_teams) == 0, bad_teams

    joined = pd.merge(transformed, nfl.import_schedules([season])[['game_id', 'season', 'week', 'away_team', 'home_team', 'away_score', 'home_score', 'result', 'spread_line']], on=['season', 'week'])

    reduced = joined[(joined['pick'] == joined['away_team']) | (joined['pick'] == joined['home_team'])]
    del joined
    assert len(reduced) == len(transformed)


    reduced['away_cover'] = reduced.apply(utils.cover_result, axis=1)
    reduced['home_cover'] = 1 - reduced['away_cover']

    reduced['pick_result'] = reduced.apply(utils.pick_result, axis=1)
    print(season, week, reduced.shape[0])
    return reduced

In [38]:
homers = pd.concat([parse_picks(szn, wk) for szn in [2022] for wk in range(1, 19)], ignore_index=True)
homers.shape

2022 1 54
2022 2 54
2022 3 54
2022 4 54
2022 5 54
2022 6 54
2022 7 48
2022 8 48
2022 9 45
2022 10 48
2022 11 48
2022 12 48
2022 13 48
2022 14 48
2022 15 48
2022 16 48


AssertionError: 

In [39]:
week = 17
season = 2022

In [40]:
df = pd.read_excel(f'../../data/betting/homers-{season}.xlsx', sheet_name=f'Wk {week}', usecols=range(14), header=1)
df = df.drop(df.index[9:])
df = df.dropna(axis=1, how='all')
if 'Notes' in df.columns:
    df = df.drop(columns=['Notes'])
df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])
df = utils.clean_df_columns(df)
df = df.rename(columns={'team': 'final', 'picks': 'pick_type'})
assert all(col in ['pick_type', 'final', 'ben', 'chuck', 'griffin', 'harry', 'hunter'] for col in df.columns)
assert len(df) == 9
df['pick_type'] = df.pick_type.map({i: 'reg' for i in range(1, 6)}).fillna(df['pick_type']).apply(str.lower)
df['pick_type'] = df.pick_type.map({'underdog': 'ud', 'survivor': 'sd'}).fillna(df['pick_type'])
df['pick_type'] = df.pick_type.apply(lambda x: 'reg' if x not in ['bb', 'reg', 'sd', 'ud', 'mnf'] else x)
df

,pick_type,final,ben,chuck,griffin,harry,hunter
0,bb,JAX,LAC,MIN,JAX,MIN,NYG
1,reg,PIT,KC,NYJ,NE,SF,JAX
2,reg,MIN,NE,PHI,BAL,NYJ,NYJ
3,reg,NYJ,CAR,JAX,LAR,MIA,PIT
4,reg,NYG,CLE,DEN,ARI,CAR,CLE
5,reg,SF,IND,CAR,WAS,ATL,CIN
6,ud,ARI,ARI,DEN,ARI,DEN,LV
7,sd,NaN,NaN,NaN,NaN,NaN,NaN
8,mnf,CIN,BUF,CIN,BUF,CIN,CIN


In [41]:
transformed = pd.melt(df, id_vars=['pick_type'], var_name='picker', value_name='pick').dropna()
transformed['season'] = season
transformed['week'] = week
transformed = transformed[transformed['pick'] != '']
# Map bad team names
transformed['pick'] = transformed['pick'].map({
    'LAR': 'LA', 
    'WSH': 'WAS', 
    'WFT': 'WAS',
    'JAG': 'JAX',
    'JAC': 'JAX',
    'PHL': 'PHI',
}).fillna(transformed['pick'])
transformed

,pick_type,picker,pick,season,week
0,bb,final,JAX,2022,17
1,reg,final,PIT,2022,17
2,reg,final,MIN,2022,17
3,reg,final,NYJ,2022,17
4,reg,final,NYG,2022,17
5,reg,final,SF,2022,17
6,ud,final,ARI,2022,17
8,mnf,final,CIN,2022,17
9,bb,ben,LAC,2022,17
10,reg,ben,KC,2022,17


In [42]:
[team for team in transformed['pick'].unique() if team not in teams]

[]

spreads are indicative of the away team

In [44]:
joined = pd.merge(transformed, nfl.import_schedules([season])[['game_id', 'season', 'week', 'away_team', 'home_team', 'away_score', 'home_score', 'result', 'spread_line']], on=['season', 'week'])

reduced = joined[(joined['pick'] == joined['away_team']) | (joined['pick'] == joined['home_team'])]
del joined
print(len(reduced), len(transformed))


reduced['away_cover'] = reduced.apply(utils.cover_result, axis=1)
reduced['home_cover'] = 1 - reduced['away_cover']

reduced['pick_result'] = reduced.apply(utils.pick_result, axis=1)
reduced

41 48


,pick_type,picker,pick,season,week,game_id,away_team,home_team,away_score,home_score,result,spread_line,away_cover,home_cover,pick_result
3,bb,final,JAX,2022,17,2022_17_JAX_HOU,JAX,HOU,31.0,3.0,-28.0,-3.5,1,0,2
29,reg,final,PIT,2022,17,2022_17_PIT_BAL,PIT,BAL,16.0,13.0,-3.0,1.0,1,0,1
42,reg,final,MIN,2022,17,2022_17_MIN_GB,MIN,GB,17.0,41.0,24.0,3.0,0,1,0
56,reg,final,NYJ,2022,17,2022_17_NYJ_SEA,NYJ,SEA,6.0,23.0,17.0,-1.0,0,1,0
66,reg,final,NYG,2022,17,2022_17_IND_NYG,IND,NYG,10.0,38.0,28.0,5.5,0,1,1
85,reg,final,SF,2022,17,2022_17_SF_LV,SF,LV,37.0,34.0,-3.0,-10.0,0,1,0
91,ud,final,ARI,2022,17,2022_17_ARI_ATL,ARI,ATL,19.0,20.0,1.0,7.0,1,0,0
133,bb,ben,LAC,2022,17,2022_17_LA_LAC,LA,LAC,10.0,31.0,21.0,6.5,0,1,2
139,reg,ben,KC,2022,17,2022_17_DEN_KC,DEN,KC,24.0,27.0,3.0,13.5,1,0,0
155,reg,ben,NE,2022,17,2022_17_MIA_NE,MIA,NE,21.0,23.0,2.0,3.0,1,0,0


In [45]:
reduced.picker.value_counts()

final      7
ben        7
chuck      7
griffin    7
harry      7
hunter     6
Name: picker, dtype: int64